# Part 2

In [213]:
#Imports Section

import networkx as nx
import numpy as np
from tqdm import tqdm
import netwulf
import random
import copy

## Exercise 1

In [214]:
#Defining the karateclub data as a function

karclub_data = nx.karate_club_graph()

In [215]:
#Setting the color of each node based on the club split

karclub_data
club_membership = nx.get_node_attributes(karclub_data, "club")

In [216]:
#setting the colors of different club nodes
colors = {}
clubs = []#For use later
mrhi_members = []
officer_members = []

for node in range(len(karclub_data.nodes)):
    if club_membership[node] == 'Mr. Hi': 
        colors[node] = "blue"
        mrhi_members.append(node)
    else:
        colors[node] = "red"
        officer_members.append(node)

clubs.append(mrhi_members)
clubs.append(officer_members)
nx.set_node_attributes(karclub_data,colors,"color")

In [217]:
#Plotting the karate club datases
netwulf.interactive.visualize(karclub_data,config={"Node color": "color"})

(None, None)

## Exercise 2

In [218]:
#Modularity function

def modularity(G, communities):
    """
    Compute the modularity for a communities of a graph.

    Parameters:
        - G: the NetworkX graph
        - communities: a list of communities, where each community is a list of node IDs

    Returns:
        - M: the modularity value for the communities
    """
    
    total_links = G.number_of_edges()
    num_of_communities = len(communities)
    links_in_community = {c: 0 for c in range(num_of_communities)}
    tot_degree = {c: 0 for c in range(num_of_communities)}

    #Calculating the number of links in each community
    for (u, v) in G.edges():
        u_c = None
        v_c = None
        for i, c in enumerate(communities):
            if u in c:
                u_c = i
            if v in c:
                v_c = i
            if u_c is not None and v_c is not None:
                break
        if u_c == v_c:
            links_in_community[u_c] += 1

    #Calculating the total degree in each community
    for u in G.nodes():
        for i, c in enumerate(communities):
            if u in c:
                tot_degree[i] += G.degree(u)
                break

    M = 0.0
    for n_c in range(num_of_communities):
        M += (links_in_community[n_c] / total_links) - ((tot_degree[n_c] / (2 * total_links)) ** 2)

    return M



## Exercise 3 : Explain in your own words the concept of modularity

The modularity formula used above tells us, in a sense, the degree to which the network is modular. Hence, it measures the difference between the number of edges within the community, and the expected number of edges given a random distribution. If the value is positibe, we see more edges in the community than we would expected at random, and vice verca for the negative value. The value ranges from -1 to 1. If it is close to one, we likely have a dense community structure, and if it is close to -1, we perhaps have a more homogenous structure without any clear structure. Modularity is a useful metric in the analysis of complex networks for this reason.

## Exercise 4 : Modularity of karate club dataset

In [219]:
modularity(karclub_data,clubs)

0.3582347140039447

## Exercise 5 : Randomization experiment, significant difference from 0

In [272]:
#Random Edge swap function

def edge_swap(G, N):
    """
    create a copy of G with N swapped edges

    Parameters:
        - G: the given graph
        - N: how many edges we want swapped

    Returns:
        - G_copy: a copy of graph g with N edges swapped at random
    """
    
    # Make a copy of the original network
    G_copy = copy.deepcopy(G)
    
    # Determine the number of edges in the graph
    num_edges = G.number_of_edges()
    
    # Perform at least N edge swaps
    edges_swapped = 0
    while edges_swapped <= N:
        # Choose two random edges
        u, v = random.sample(karclub_data.edges(), 1)[0]
        x, y = random.sample(karclub_data.edges(), 1)[0]
        # Check that the edges are distinct
        if u != v and v != x:
            # Check if the edges (u,y) and (x,v) exist
            if not G_copy.has_edge(u, y) and not G_copy.has_edge(x, v):
                # Swap the edges
                print(edges_swapped)
                try: 
                    G_copy.remove_edge(u, v)
                    G_copy.remove_edge(x, y)
                    G_copy.add_edge(u, y)
                    G_copy.add_edge(x, v)
                except:
                    print(u,v,x,y)
                    print(G_copy.edges())
                #Counter for number of swaps
                edges_swapped += 1

    return G_copy

## Exercise 6 : Double Check Algorithm

In [275]:
edge_swap(karclub_data, 100)

0
1
2
3
4
5
6
28 31 23 29
[(0, 1), (0, 2), (0, 4), (0, 5), (0, 6), (0, 7), (0, 8), (0, 10), (0, 11), (0, 17), (0, 19), (0, 21), (0, 31), (0, 29), (0, 32), (0, 33), (1, 2), (1, 3), (1, 7), (1, 13), (1, 17), (1, 19), (1, 21), (1, 30), (2, 3), (2, 7), (2, 8), (2, 13), (2, 27), (2, 28), (2, 32), (2, 33), (3, 7), (3, 12), (3, 13), (3, 8), (4, 6), (4, 33), (5, 6), (5, 10), (5, 16), (6, 32), (8, 30), (8, 33), (9, 33), (9, 27), (10, 14), (12, 23), (13, 33), (13, 22), (14, 32), (15, 32), (15, 33), (16, 20), (18, 32), (18, 33), (19, 33), (20, 33), (22, 32), (23, 25), (23, 27), (23, 32), (23, 33), (24, 25), (24, 27), (24, 31), (25, 31), (26, 29), (26, 33), (28, 33), (29, 32), (29, 33), (30, 32), (30, 33), (31, 32), (31, 33), (32, 33)]
7
8
9
5 6 22 33
[(0, 1), (0, 2), (0, 4), (0, 6), (0, 7), (0, 8), (0, 10), (0, 11), (0, 17), (0, 19), (0, 21), (0, 31), (0, 29), (0, 32), (0, 33), (0, 13), (1, 2), (1, 3), (1, 7), (1, 13), (1, 17), (1, 19), (1, 21), (1, 25), (2, 3), (2, 7), (2, 8), (2, 13), (2, 27), 

In [234]:
karclub_data.edges()

EdgeView([(0, 1), (0, 2), (0, 3), (0, 4), (0, 5), (0, 6), (0, 7), (0, 8), (0, 10), (0, 11), (0, 12), (0, 13), (0, 17), (0, 19), (0, 21), (0, 31), (1, 2), (1, 3), (1, 7), (1, 13), (1, 17), (1, 19), (1, 21), (1, 30), (2, 3), (2, 7), (2, 8), (2, 9), (2, 13), (2, 27), (2, 28), (2, 32), (3, 7), (3, 12), (3, 13), (4, 6), (4, 10), (5, 6), (5, 10), (5, 16), (6, 16), (8, 30), (8, 32), (8, 33), (9, 33), (13, 33), (14, 32), (14, 33), (15, 32), (15, 33), (18, 32), (18, 33), (19, 33), (20, 32), (20, 33), (22, 32), (22, 33), (23, 25), (23, 27), (23, 29), (23, 32), (23, 33), (24, 25), (24, 27), (24, 31), (25, 31), (26, 29), (26, 33), (27, 33), (28, 31), (28, 33), (29, 32), (29, 33), (30, 32), (30, 33), (31, 32), (31, 33), (32, 33)])

## Exercise 7 : 1000 randomized 